## Julia basics

- https://learnxinyminutes.com/docs/julia/

### Numeric Types


#### Arrays

You can define an array by using:

- a = [1,2,3,4]

In [2]:
a = [1,2,3,4,5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [3]:
length(a)

5

In [7]:
for k=1

LoadError: LoadError: syntax: incomplete: premature end of input
while loading In[7], in expression starting on line 1

In [13]:
a = [x for x in 1:1000];

In [15]:
length(a)

1000

In [2]:
append!(a,23);

LoadError: LoadError: UndefVarError: a not defined
while loading In[2], in expression starting on line 1

In [36]:
append!(a,-123)
length(a)

1000015

In [ ]:
mem

In [21]:
# Generate an array of length 1000000
n = 1000000
a = [1]

# julia arrays start at position 1 
for k = 2:n
    append!(a,k)
end

In [37]:
length(a)

1000015

In [39]:
3+3

6

In [480]:
+ (3,3,3)

9

# Functions

In [514]:
function op(x,y)
    return x+y
end

op (generic function with 1 method)

In [515]:
print( "first call: ", op(3,3), "\nsecond call: ", op(5.5,4.3))

first call: 6
second call: 9.8

#### Passing vectors to functions

If a function modifies the first argument passed it is a convention to write an exclamation mark
at the end of the name of the funcion.

For example, let us build a function that modifies the incoming vector and writes 10 at every even position.

In [529]:
workspace()
function modificar_posiciones_par!(x)
    n = length(x)
    
    for k=1:n
        if k%2==0
            x[k] = 10
        end
    end
    return x
end

modificar_posiciones_par! (generic function with 1 method)

In [538]:
x = Vector(1:20);
x'

1×20 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20

In [539]:
modificar_posiciones_par!(x);

In [541]:
x'

1×20 Array{Int64,2}:
 1  10  3  10  5  10  7  10  9  10  11  10  13  10  15  10  17  10  19  10

In [542]:
workspace()
function modificar_posiciones_par(x)
    n = length(x)
    x_copy = copy(x)
    
    for k=1:n
        if k%2==0
            x_copy[k] = 10
        end
    end
    return x_copy
end

modificar_posiciones_par (generic function with 1 method)

In [543]:
x = Vector(1:20);
x'

1×20 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20

In [544]:
x_modified = modificar_posiciones_par(x);

In [545]:
x'

1×20 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20

In [546]:
x_modified'

1×20 Array{Int64,2}:
 1  10  3  10  5  10  7  10  9  10  11  10  13  10  15  10  17  10  19  10

# Defining your own type

Types can be thought as boxes containing fields. Every field has a name and it is highly recoomended to have a type as well (but not required)



In [30]:
workspace()
type troll{T}
    weight::T
    height::T
end

In [31]:
a_troll = troll(3,4)

troll{Int64}(3,4)

In [39]:
a_troll

troll{Float64}(3.0,4.0)

WE can use other types to define a troll

In [40]:
a_troll = troll(3., 4.)

troll{Float64}(3.0,4.0)

Nevertheless we defined ty type stating that every filed had to be form the same type. Therefore troll(3,4.) will not work

In [72]:
a_troll = troll(3,4.)

LoadError: LoadError: UndefVarError: troll not defined
while loading In[72], in expression starting on line 1

### Creating inner constructors

We can add a function inside a type used to instanciate the type.

This function can be used for example in order to..

- Put restrictions on the type, such as..
     - not allow negative values 
     - Not allow 'rare' characters

In [372]:
workspace()

In [373]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h, i)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w,h,i)
    end
end

In [374]:
methodswith(Giant)

0-element Array{Method,1}

Notice that if a constructor is defined inside a type we must explicitly define a way to instanciate new elements for every possible type T.

In [375]:
Giant(3.,4.,"low")

LoadError: LoadError: MethodError: no method matching Giant{T}(::Float64, ::Float64, ::String)
Closest candidates are:
  Giant{T}{T}(::Any) at sysimg.jl:53
while loading In[375], in expression starting on line 1

Let us do it

In [376]:
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

#Giant{T}(weight::T, height::T) = Giant{T}(weight, height ; intelligence="low")

Giant{T}

In [377]:
methodswith(Giant)

0-element Array{Method,1}

In [378]:
giant = Giant(3., 4., "low")

Giant{Float64}(3.0,4.0,"low")

In [379]:
giant.height, giant.weight, giant.intelligence

(4.0,3.0,"low")

Notice that, sice we have written a restriction on the height of the giant
we cannot instanciate giants with less than 2 meter height

In [380]:
giant = Giant(3.,4.,"medium")

Giant{Float64}(3.0,4.0,"medium")

In [381]:
giant.height, giant.weight, giant.intelligence

(4.0,3.0,"medium")

In [382]:
giant = Giant(3.,1.,"medium")

LoadError: LoadError: ArgumentError: Giants are taller

while loading In[382], in expression starting on line 1

### Assigning  a default value to a type 

By default we might think that Giants are quite stupid and therefore we might be interested into assigning "stupid" intelligence as default.

Notice that default values in Julia are defined after ```;```

This can be specially handly in order to provide default values for types, for example when there could be 'standard values' for some of the fileds inside a type and we do not want to bother the user selecting field values. 

Using this trick we can allow the user to set values to all the fields the user wants to but at the same time, allow the user **not to** define all possible fields .

In [461]:
workspace()

In [462]:
type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h ,  intelligence)
    end
    
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)


Giant{T}

In [463]:
giant = Giant(3.,5.)

Giant{Float64}(3.0,5.0,"stupid")

In [464]:
giant.weight, giant.height, giant.intelligence

(3.0,5.0,"stupid")

In [465]:
giant.intelligence = "smart"

"smart"

In [466]:
giant.weight, giant.height, giant.intelligence

(3.0,5.0,"smart")

Notice that we cannot pass now a value for the field intelligence

In [467]:
rare_giant = Giant(5.,4.,"smart")

LoadError: LoadError: MethodError: no method matching Giant{T}(::Float64, ::Float64, ::String)
Closest candidates are:
  Giant{T}{T}(::T, ::T; intelligence) at In[462]:16
  Giant{T}{T}(::Any) at sysimg.jl:53
while loading In[467], in expression starting on line 1

Notice that we cannot use this definition since we need to specify how to create a Giant  when we pass the intelligence field.

### Allowing types to have fields with defeault values

https://groups.google.com/forum/#!topic/julia-users/9jM7GIoh7YY

In [475]:
workspace()

type Giant{T}
    weight::T
    height::T
    intelligence::String
    
    function Giant(w, h ; intelligence="stupid")
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
    
    function Giant(w, h, intelligence)
        if h<2
            throw(ArgumentError("Giants are taller\n"))
        end
        return new(w, h, intelligence)
    end
end

## IMPORTANT TO ADD CONSTRUCTOR FOR EVERY TYPE T !!!!
Giant{T}(weight::T, height::T) = Giant{T}(weight, height)
Giant{T}(weight::T, height::T, intelligence::String) = Giant{T}(weight, height, intelligence)

Giant{T}

In [477]:
rare_giant = Giant(5., 4., "smart")

Giant{Float64}(5.0,4.0,"smart")

In [478]:
giant = Giant(3., 5.)

Giant{Float64}(3.0,5.0,"stupid")

#### Type man


The following example is the type man that contains 3 fields, **```heigh```**,**``` weight```** and **```name```**.

In [171]:
type man{T}
    height::T
    weight::T
    name::String
end

A type can be instanciated by assigning values to its fields, the fields of a type can be checked with the **```fieldnames```** function

In [172]:
david = man(183, 80, "David")

man{Int64}(183,80,"David")

In [173]:
fieldnames(david)

3-element Array{Symbol,1}:
 :height
 :weight
 :name  

Notice that **you cannot specify a particular value on a field**.

In [18]:
type girl{T}
    height::T
    weight::T
    name="Julia"
end

LoadError: LoadError: syntax: "name="Julia"" inside type definition is reserved
while loading In[18], in expression starting on line 1

Nevertheless you can define a type and a (so called) inner constructor which might specify a particular value of the type.

In [40]:
type girl2{T}
    height::T
    weight::T
    name::String
end

In [42]:
girl2(23, 123, "julia")

girl2{Int64}(23,123,"julia")

In [44]:
typeof(girl2)

DataType

In [45]:
methodswith(girl2)

0-element Array{Method,1}

### Inner constructor

In [53]:
type girl4{T}
    height::T
    weight::T
    name::String
    
    function girl4(height, weight, name)
        new(23,12,"lala")
    end
end

In [54]:
girl4(23,123,"julia")

LoadError: LoadError: MethodError: no method matching girl4{T}(::Int64, ::Int64, ::String)
Closest candidates are:
  girl4{T}{T}(::Any) at sysimg.jl:53
while loading In[54], in expression starting on line 1

### The importance of declaring code inside functions

In [55]:
@time begin
pos = 0
num_steps = 10^4
numwalkers = 10^4
final_square_positions = Int[]

for i in 1:numwalkers
    for j in 1:num_steps
        pos += ifelse( rand() < 0.5, -1, +1)
    end
        push!(final_square_positions, pos^2)
end
    
end

 15.190851 seconds (380.49 M allocations: 7.160 GB, 4.46% gc time)


In [57]:
function fool_function()
    pos = 0
    num_steps = 10^4
    numwalkers = 10^4
    final_square_positions = Int[]

    for i in 1:numwalkers
        for j in 1:num_steps
            pos += ifelse( rand() < 0.5, -1, +1)
        end
            push!(final_square_positions, pos^2)
    end
end

fool_function (generic function with 1 method)

In [59]:
@time fool_function()

  0.299746 seconds (18 allocations: 256.859 KB)
